In [1]:
import os

import scipy.io as sio
import scipy.io.wavfile as wav

import numpy as np
import matplotlib.pyplot as plt

import IPython.display

In [15]:
filtresFns = ["subject_003/hrir_final.mat", "subject_065/hrir_final.mat"]
filtreDict = sio.loadmat(filtresFns[0])
filtreDroit = filtreDict["hrir_r"]
filtreGauche = filtreDict["hrir_l"]

In [16]:
# for key in filtreDict:
#     if "__" in key: continue
#     print(key, "\t%10s" % " ", type(filtreDict[key]), filtreDict[key].shape)

# az = 0
# plt.figure(figsize=(5, 5))
# plt.subplot(121)
# plt.imshow(filtreDict["hrir_l"][az, :, :].T, cmap="gray")
# plt.title("filtre gauche, az=%i" % az)
# plt.subplot(122)
# plt.imshow(filtreDict["hrir_r"][az, :, :].T, cmap="gray")
# plt.title("filtre droit, az=%i" % az)

# plt.tight_layout()
# plt.show()

In [17]:
sonsFns = os.listdir("Sons/")
print(sonsFns)

['Cello_pizz.wav', 'Glass_break_44.wav', 'Cello_arco.wav', 'C4.wav', 'C3.wav', 'snare1_44.wav', 'drum_roll44.wav']


In [18]:
sonFn = "Sons/%s" % sonsFns[-2]
print(sonFn)

fs, signal = wav.read(sonFn)
IPython.display.Audio(signal, rate=fs)

Sons/snare1_44.wav


In [19]:
elmax = 50
elevations = -45 + 5.625 * np.arange(elmax)
azimuths = np.concatenate(([-80, -65, -55], np.arange(-45, 50, 5), [55, 65, 80]))


def getNearestUCDpulse(azimuth, elevation, filtr):

    # Find closest index for elevation in table
    el = int(np.round(((elevation + 45) / 5.625)))
    el = min(elmax, max(el, 1))
    elerr = el - (elevation + 45) / 5.625
    
    az = np.abs(azimuths - azimuth).argmin()
    azerr = np.abs(azimuths - azimuth)[az]
    
    return filtr[az, el, :], elerr, azerr

def spatialSynth(az, el, fg, fd):
    fl, elerr, azerr = getNearestUCDpulse(az, el, fg)
    fr, elerr, azerr = getNearestUCDpulse(az, el, fd)
    leftSig = np.convolve(signal, fl).reshape(1, -1)
    rightSig = np.convolve(signal, fr).reshape(1, -1)

    return np.vstack((leftSig, rightSig))


az, el = 80, 0
binaural = spatialSynth(az, el, filtreGauche, filtreDroit)
IPython.display.Audio(binaural, rate=fs)

The results are good across all samples, with perhaps less convincing results on the `cello_pizz` file.

In [20]:
song = []
for az in azimuths:
    binaural = spatialSynth(az, el, filtreGauche, filtreDroit)
    song.append(binaural)
song = np.hstack(song)
IPython.display.Audio(song, rate=fs)

In order to make a continuous effect we'd do an interpolation over two contiguous filters, e.g. to get a filter at -60 we'd mix the filters at -65 and -55.

In [42]:
def spatialContinuousSynth(az, el, fg, fd):
    
    azerr = np.abs(azimuths - az).min()
    
    if azerr == 0:
        return spatialSynth(az, el, fg, fd)
    
    azix = np.abs(azimuths - az).argmin()

    alpha = np.abs(azerr) / (azimuths[azix] - azimuths[azix + np.sign(azerr)])
    fl, elerr, azerr = getNearestUCDpulse(azimuths[azix], el, fg)
    fr, elerr, azerr = getNearestUCDpulse(azimuths[azix], el, fd)

    upl = alpha * np.convolve(signal, fl).reshape(1, -1)
    upr = alpha * np.convolve(signal, fr).reshape(1, -1)
    up = np.vstack((upl, upr))

    fl, elerr, azerr = getNearestUCDpulse(azimuths[azix + np.sign(azerr)], el, fg)
    fr, elerr, azerr = getNearestUCDpulse(azimuths[azix + np.sign(azerr)], el, fd)

    downl = (1 - alpha) * np.convolve(signal, fl).reshape(1, -1)
    downr = (1 - alpha) * np.convolve(signal, fr).reshape(1, -1)
    down = np.vstack((upl, upr))

    return alpha * up + (1 - alpha) * down

closestUp = spatialSynth(65, el, filtreGauche, filtreDroit)
continuous = spatialContinuousSynth(63, el, filtreGauche, filtreDroit)
closestDown = spatialSynth(55, el, filtreGauche, filtreDroit)

print("up")
IPython.display.display(IPython.display.Audio(closestUp, rate=fs))
print("interpolate")
IPython.display.display(IPython.display.Audio(continuous, rate=fs))
print("down")
IPython.display.display(IPython.display.Audio(closestDown, rate=fs))

print("all")
IPython.display.display(IPython.display.Audio(
    np.hstack((closestUp, continuous, closestDown)), rate=fs))

up


interpolate


down


all
